In [ ]:
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt

# 데이터 로딩

In [ ]:
trainset_raw = torchvision.datasets.MNIST(root='./data', train=True, download=True)
testset_raw = torchvision.datasets.MNIST(root='./data', train=False, download=True)

In [ ]:
for row in trainset_raw.data[0].numpy():
    print(" ".join(f"{v:3}" for v in row))

In [ ]:
# numpy → tensor 변환 & 정규화
def preprocess_mnist(dataset):
    images = dataset.data.float() / 255.0  # [0,255] → [0,1]
    mean, std = images.mean() , images.std()
    # images = (images - mean) / std         # Normalize
    labels = dataset.targets
    return images, labels

In [ ]:
train_images, train_labels = preprocess_mnist(trainset_raw)
test_images, test_labels = preprocess_mnist(testset_raw)

In [ ]:
# Dataset 객체로 묶기
full_dataset = torch.utils.data.TensorDataset(train_images, train_labels)

In [ ]:
# train/val split (Dataset을 그대로 분할)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# 모델 생성

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
model = DNN().to(device)  # 모델을 GPU로 이동
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습

In [ ]:
train_losses, val_losses, val_accuracies = [], [], []

In [ ]:
for epoch in range(20):
    model.train()
    running_loss = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_losses.append(running_loss / len(trainloader))

    # ----- Validation -----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for images, labels in valloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_losses.append(val_loss / len(valloader))
    val_accuracies.append(100 * correct / total)

    print(f"Epoch [{epoch+1}/20], Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {val_accuracies[-1]:.2f}%")

# 학습결과 확인

In [ ]:
plt.figure()
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.legend()
plt.title("Loss")
plt.show()

In [ ]:
plt.figure()
plt.plot(val_accuracies, label="Val Accuracy")
plt.legend()
plt.title("Accuracy")
plt.show()